In [2]:
!pip install kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"yashkumar2412","key":"110ff56020b3663dabc0301d71021a49"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!ls /root/

In [5]:
!kaggle competitions download -c ieee-fraud-detection

ieee-fraud-detection.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
!unzip ieee-fraud-detection.zip

Archive:  ieee-fraud-detection.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [8]:
!python -m pip install -q --upgrade pip
!python -m pip install -q autogluon

In [9]:
!pip install -q --upgrade pyarrow

In [2]:
!ls /content/

 AutoGluonModels	    kaggle.json		    test_identity.csv	   train_transaction.csv
 ieee-fraud-detection.zip   sample_data		    test_transaction.csv
'kaggle (1).json'	    sample_submission.csv   train_identity.csv


In [3]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

directory = '/content/' # directory where you have downloaded the data CSV files from the competition
label = 'isFraud'  # name of target variable to predict in this competition
eval_metric = 'roc_auc'  # Optional: specify that competition evaluation metric is AUC
save_path = directory + 'AutoGluonModels/'  # where to store trained models

train_identity = pd.read_csv(directory+'train_identity.csv')
train_transaction = pd.read_csv(directory+'train_transaction.csv')

In [4]:
train_data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
train_data.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [5]:
train_data.shape

(590540, 434)

In [6]:
train_data = train_data[:10000] # subset of training dataset to avoid 'Out of memory' error
train_data.shape

(10000, 434)

In [9]:
predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path, verbosity=3).fit(
    train_data, presets='good_quality', time_limit=120
)

results = predictor.fit_summary()

Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
GPU Count:          1
Memory Avail:       2.66 GB / 12.67 GB (21.0%)
Disk Space Avail:   65.27 GB / 112.64 GB (57.9%)
Presets specified: ['good_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': True,
 'num_bag_sets': 1,
 'refit_full': True,
 'save_bag_folds': False,
 'set_best_to_refit_full': True}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': True,
 'calibrate': 'auto',
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
             'enable_ray_logging': True,
             'holdout_data': None,
             'holdout_frac': 0.111111

*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val eval_metric  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0         LightGBMXT_BAG_L1   0.885301     roc_auc       0.527672  43.505905                0.527672          43.505905            1      False          1
1       WeightedEnsemble_L2   0.885301     roc_auc       0.532360  43.547110                0.004689           0.041205            2      False          2
2       WeightedEnsemble_L3   0.885301     roc_auc       0.534097  43.537017                0.006426           0.031112            3      False          3
3  WeightedEnsemble_L3_FULL        NaN     roc_auc            NaN   4.152821                     NaN           0.031112            3       True          6
4  WeightedEnsemble_L2_FULL        NaN     roc_auc            NaN   4.162914                     NaN           0.041205            2       True          5
5    Lig

In [ ]:
test_identity = pd.read_csv(directory+'test_identity.csv')
test_transaction = pd.read_csv(directory+'test_transaction.csv')
test_data = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

y_predproba = predictor.predict_proba(test_data)
y_predproba.head(5)  # some example predicted fraud-probabilities

In [ ]:
predictor.positive_class

In [ ]:
predictor.class_labels  # classes in this list correspond to columns of predict_proba() output

In [ ]:
y_predproba = predictor.predict_proba(test_data, as_multiclass=False)

In [ ]:
submission = pd.read_csv(directory+'sample_submission.csv')
submission['isFraud'] = y_predproba
submission.head()
submission.to_csv(directory+'my_submission.csv', index=False)